In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torch import nn

from tqdm import tqdm
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cuda device


In [2]:
data_train = pd.read_csv("digit-recognizer/train.csv")
data_test = pd.read_csv("digit-recognizer/test.csv")

In [3]:
data_train.head()
data_test.head()

y_train = data_train["label"]

In [4]:
labels = y_train.unique().sort()
print(labels)

None


In [5]:
X_train = data_train.drop("label", axis = 1)
X_test = data_test

In [6]:
knn = KNeighborsClassifier(n_neighbors=10)

In [7]:
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=10)

In [8]:
#KNN_predict = knn.predict(X_test)
#KNN_predict

In [9]:
class NeuralNetwork(nn.Module):
    def __init__(self, embedding_size, count_labels):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(embedding_size, 1024),
            nn.ReLU(),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, count_labels),
            nn.Softmax(dim=1)
        )
        
    def forward(self, x):
        x = self.linear_relu_stack(x)
        #return torch.argmax(x, dim = 1)
        return x

In [10]:
class MNIST_Train_Dataset(Dataset):
    def __init__(self, mnist, labels):
        self.mnist = mnist
        self.labels = labels
    
    def __len__(self):
        return len(self.mnist)
    
    def __getitem__(self, idx):
        return self.mnist[idx], self.labels[idx]
    
class MNIST_valid_Dataset(Dataset):
    def __init__(self, mnist):
        self.mnist = mnist
    
    def __len__(self):
        return len(self.mnist)
    
    def __getitem__(self, idx):
        return self.mnist[idx]

In [11]:
train_tensor = torch.tensor(X_train.values, dtype = torch.float).to(device)
labels_tensor = torch.tensor(y_train).to(device)

test_tensor = torch.tensor(X_test.values, dtype = torch.float).to(device)

train_dataset = MNIST_Train_Dataset(train_tensor, labels_tensor)
test_dataset = MNIST_valid_Dataset(test_tensor)

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=True)

In [12]:
embedding_size = X_train.shape[1]
model = NeuralNetwork(embedding_size, 10).to(device)

In [13]:
num_epochs = 2

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [14]:
def normalize(x, Min=0, Max=9):
    return (x - Min) / (Max - Min)

In [15]:
for epoch in range(num_epochs):
    print("Epoch N°{}".format(epoch))
    LOSS = 0
    for idx, (data, label) in enumerate(tqdm(train_dataloader)):
        pred = model(data)

        loss = loss_fn(pred, label)
        LOSS += loss
        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

    print("Loss: {}".format(LOSS/X_train.shape[0]))

Epoch N°0


100%|███████████████████████████████████████████████████████████████| 5250/5250 [00:12<00:00, 420.86it/s]


Loss: 0.22108569741249084
Epoch N°1


100%|███████████████████████████████████████████████████████████████| 5250/5250 [00:12<00:00, 420.66it/s]


Loss: 0.19227007031440735


In [16]:
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)

print(input)
print(target)
output = loss(input, target)
output.backward()

# Example of target with class probabilities
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5).softmax(dim=1)
output = loss(input, target)
output.backward()

tensor([[ 0.6588, -1.1348, -0.8381, -2.2784, -0.6839],
        [-0.1733,  0.2359, -0.2663, -2.5456, -1.8658],
        [ 0.0489, -0.6432, -0.6507,  0.3372,  0.0722]], requires_grad=True)
tensor([3, 3, 3])


In [17]:
X, y=next(iter(train_dataloader))

In [18]:
val = model(X)
result = torch.argmax(val, dim = 1)
print(val)
print(result)
print(y)

tensor([[7.2035e-11, 9.3152e-10, 1.7273e-13, 7.4093e-10, 9.1358e-09, 2.5048e-08,
         4.5772e-13, 1.0000e+00, 1.5405e-12, 1.7756e-06],
        [5.6225e-11, 1.0719e-07, 9.5170e-10, 8.2263e-09, 5.9568e-05, 2.9116e-08,
         5.0698e-12, 3.3487e-08, 1.2725e-06, 9.9994e-01],
        [1.4147e-18, 6.7881e-17, 2.4620e-14, 1.3955e-21, 1.0000e+00, 6.9958e-15,
         3.9845e-15, 1.2582e-14, 4.4470e-20, 3.1670e-14],
        [5.4103e-05, 1.3676e-05, 6.9164e-07, 1.0480e-05, 3.5135e-02, 9.8149e-03,
         7.8145e-06, 9.4766e-01, 1.0473e-06, 7.3053e-03],
        [3.5495e-07, 1.2929e-14, 4.3157e-11, 4.9695e-09, 1.0124e-08, 1.0000e+00,
         3.9804e-07, 2.8078e-09, 3.6590e-09, 7.7996e-11],
        [6.4066e-11, 1.9587e-12, 1.0000e+00, 9.9720e-11, 4.1182e-14, 4.2781e-16,
         1.0766e-12, 4.7056e-16, 4.4998e-15, 1.3126e-17],
        [6.2599e-08, 5.4010e-09, 1.2755e-07, 1.7897e-11, 1.0367e-04, 5.3497e-07,
         9.9990e-01, 2.4046e-10, 1.0279e-09, 9.9027e-13],
        [2.7132e-10, 6.5997